In [1]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [2]:
# !pip install --upgrade transformers
# !pip install tf-keras

In [ ]:
# !pip install transformers==4.37.2

In [4]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [6]:
# https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k
data_test = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype= str)
data_train = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)

In [7]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [8]:
print(data_train.shape)
print(data_test.shape)

(25000, 2)
(25000, 2)


In [ ]:
# DistilBERT model
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment',
                                           val_df = data_test, maxlen = 400, preprocess_mode = 'distilbert')

In [10]:
model = text.text_classifier(name='distilbert', train_data=train, preproc=preproc)

Is Multi-Label? False
maxlen is 400
done.


In [11]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 16)

In [12]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
1563/1563 [==============================] - 1454s 916ms/step - loss: 0.3044 - accuracy: 0.8718 - val_loss: 0.1642 - val_accuracy: 0.9406
Epoch 2/2
1563/1563 [==============================] - 1431s 916ms/step - loss: 0.1614 - accuracy: 0.9408 - val_loss: 0.0796 - val_accuracy: 0.9774


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('model/distilbert')

In [14]:
# prediction
data = ['this movie was really bad. acting was also bad. I will not watch again',
        'the movie was really great. I will see it again', 'another great movie. must watch to everyone']

predictor.predict(data)

['neg', 'pos', 'pos']

In [15]:
predictor.predict(data, return_proba=True)

array([[0.990303  , 0.009697  ],
       [0.00835962, 0.9916403 ],
       [0.00750088, 0.9924992 ]], dtype=float32)